In [ ]:
# CONFIGURANDO ARMAZENAMENTO
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# LOCAL DE EXECUÇÃO
%cd /content/drive/MyDrive/UFRN/colab/

In [230]:
# BIBLIOTECAS
import pandas as pd
import numpy as np
import requests as rq
import re
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns

In [228]:
# LER ARQUIVO CSV
try:
  df_empregos = pd.read_csv('./arquivo_empregos.csv')
except:
  empregos = []

In [229]:
# GRAVAR ARQUIVO CSV
df_empregos.to_csv('./arquivo_empregos.csv',index=False)

coleta de dados

In [231]:
# COLETANDO DADOS DO SITE E FAZENDO A RASPAGEM DE DADOS 1º ETAPA
# NÃO USE ESSA OPÇÃO QUANDO ESTIVER CARREGANDO OS DADOS PELO DATASET
# NESSE CASO PULE DIRETO PARA 2º ETAPA

# obtendo dados pelo site
url1 = 'https://www.trabalhabrasil.com.br/vagas-empregos-em-natal-rn?ordenacao=1'
url2 = 'https://www.trabalhabrasil.com.br/vagas-empregos-em-joao-pessoa-pb'
url3 = 'https://www.trabalhabrasil.com.br/vagas-empregos-em-fortaleza-ce'
url4 = 'https://www.trabalhabrasil.com.br/vagas-empregos-em-recife-pe'
url5 = 'https://www.trabalhabrasil.com.br/vagas-empregos-em-salvador-ba'
url6 = 'https://www.trabalhabrasil.com.br/vagas-empregos-em-sao-luis-ma'
url7 = 'https://www.trabalhabrasil.com.br/vagas-empregos-em-teresina-pi'
url8 = 'https://www.trabalhabrasil.com.br/vagas-empregos-em-aracaju-se'
url9 = 'https://www.trabalhabrasil.com.br/vagas-empregos-em-maceio-al'
vet_url = [url1,url2,url3,url4,url5,url6,url7,url8,url9]
# criação das listas vazias
empregos = []
quant_pesquisa = np.size(vet_url)
# variavel usada para informa o valor de um salario minimo
minimo = 1100.00
for i2 in range(quant_pesquisa):
  url = vet_url[i2]
  # obtendo cidade
  cidade = url.split('em-')
  cidade = cidade[1].split('?')
  cidade = cidade[0]
  #Obter dados html da url
  response = rq.get(url)
  #Cria objeto BeautifulSoup com analisador de HTML (parser)
  pagina_html = BeautifulSoup(response.text,'html.parser')
  pagina2 = pagina_html.find('div',{'id': 'jobs-wrapper'})
  pagina3 = pagina2.findAll('div',{'class':'jg__job'})
  for i in range(60):
    vaga = pagina3[i].find('a')
    url_vaga = vaga.get('href')
    # obtendo a codigo do anuncio retirado da url
    codigo_anuncio = url_vaga.split('/')
    tamanho = len(codigo_anuncio)
    codigo_anuncio = codigo_anuncio[tamanho -1]
    codigo_anuncio = int(codigo_anuncio)
    job_r = vaga.find('div',{'class': 'job__header'})
    job_i = job_r.find('div',{'class': 'job__info'})
    nome = job_i.find('h3').get_text()
    instituicao = job_i.find('h4').get_text()
    job_d = vaga.find('div',{'class': 'job__details'})
    div = job_d.find('div')
    salario =  div.find('h5').get_text()
    # verifica se na string salario foi encontrado o caractere R$
    # se for significa que o empregador informa o salario e esse valor vai ser armazenado
    # se não for as informações sobre a vaga será amarzenada na lista empregos_n essa lista
    # armazena as vagas sem informação de salario
    verificar_salario = salario.count('R$ ')
    faixa_salario = 0
    # o salario é recebido como string no formato: R$ 1.000,00
    # os metodos replace removem e alteram caracteres ficando no formato: 1000.00
    if(verificar_salario == 1):
      salario = salario.replace('R$ ','')
      salario = salario.replace('.','')
      salario = salario.replace(',','.')
      # converte de string para float
      salario = float(salario)
      if(salario >0 and salario <=minimo):
        faixa_salario = 1
      elif(salario >minimo and salario <= minimo*2):
        faixa_salario = 2
      elif(salario >minimo*2 and salario <=minimo*3):
        faixa_salario = 3
      elif(salario > minimo*3 and salario <= minimo*4):
        faixa_salario = 4
      elif(salario > minimo*4 and salario <= minimo*5):
        faixa_salario = 5
      elif(salario > minimo*5):
        faixa_salario = 6
    else:
      salario = 0
    opcao = {'vaga': nome,'remuneracao':salario,'companhia':instituicao,'id':codigo_anuncio,'faixa_salarial':faixa_salario,'cidade':cidade}
    empregos.append(opcao)
# criar o dataframe
df_empregos = pd.DataFrame(empregos)
# removendo resultado sem informações de salario e com valores abaixo de 400
# por que o menor valor sarial é de meio salario se a vaga for para aprendiz valores
# abaixo disso podem ser erro de digitação
df_empregos = df_empregos[df_empregos['remuneracao'] > 400]

In [232]:
# FAZENDO RASPAGEM DE DADOS 2º ETAPA

# somando todos os valores da coluna remuneracao de cada cidade selecionada
r_natal = df_empregos[df_empregos['cidade'] == 'natal-rn'].loc[:,'remuneracao'].sum()
r_joaopessoa = df_empregos[df_empregos['cidade'] == 'joao-pessoa-pb'].loc[:,'remuneracao'].sum()
r_fortaleza = df_empregos[df_empregos['cidade'] == 'fortaleza-ce'].loc[:,'remuneracao'].sum()
r_recife = df_empregos[df_empregos['cidade'] == 'recife-pe'].loc[:,'remuneracao'].sum()
r_salvador = df_empregos[df_empregos['cidade'] == 'salvador-ba'].loc[:,'remuneracao'].sum()
r_saoluiz = df_empregos[df_empregos['cidade'] == 'sao-luis-ma'].loc[:,'remuneracao'].sum()
r_terezina = df_empregos[df_empregos['cidade'] == 'teresina-pi'].loc[:,'remuneracao'].sum()
r_aracaju = df_empregos[df_empregos['cidade'] == 'aracaju-se'].loc[:,'remuneracao'].sum()
r_maceio = df_empregos[df_empregos['cidade'] == 'maceio-al'].loc[:,'remuneracao'].sum()
df_soma_salario = pd.DataFrame({ 'cidade':['natal','joao_pessoa','fortaleza','recife','salvador','sao_luiz','terezina','aracaju','maceio'],'valor':[r_natal,r_joaopessoa,r_fortaleza,r_recife,r_salvador,r_saoluiz,r_terezina,r_aracaju,r_maceio]})
# verificando o numero de vagas de emprego com informações de salario para cada cidade
q_natal=len(df_empregos[df_empregos['cidade'] == 'natal-rn'].loc[:,'cidade'])
q_joaopessoa=len(df_empregos[df_empregos['cidade'] == 'joao-pessoa-pb'].loc[:,'cidade'])
q_fortaleza=len(df_empregos[df_empregos['cidade'] == 'fortaleza-ce'].loc[:,'cidade'])
q_recife=len(df_empregos[df_empregos['cidade'] == 'recife-pe'].loc[:,'cidade'])
q_salvador=len(df_empregos[df_empregos['cidade'] == 'salvador-ba'].loc[:,'cidade'])
q_saoluiz=len(df_empregos[df_empregos['cidade'] == 'sao-luis-ma'].loc[:,'cidade'])
q_teresina=len(df_empregos[df_empregos['cidade'] == 'teresina-pi'].loc[:,'cidade'])
q_aracaju=len(df_empregos[df_empregos['cidade'] == 'aracaju-se'].loc[:,'cidade'])
q_maceio=len(df_empregos[df_empregos['cidade'] == 'maceio-al'].loc[:,'cidade'])
quant_amostra_cidade = {'natal':q_natal,'joao_pessoa':q_joaopessoa,'fortaleza':q_fortaleza,'recife':q_recife,'salvador':q_salvador,'sao_luiz':q_saoluiz,'terezina':q_teresina,'aracaju':q_aracaju,'maceio':q_maceio }
#criando dataframe a partir da lista quant_total
df_quant_amostra_cidade = pd.DataFrame([quant_amostra_cidade])

Analise exploratoria

In [ ]:
# exibir dataframe
df_empregos

In [ ]:
# descrição estatistica de todas as colunas numerícas do dataframe
df_empregos.describe()

In [ ]:
# exibir a quantidade total de amostras validas em cada cidade
df_quant_amostra_cidade

In [ ]:
# grafico em colunas com a quantidade de amostras validas de cada cidade separadamente
df_quant_amostra_cidade.plot(figsize=(10,5),kind='bar',title='Quantidade de amostras por cidade')

In [ ]:
# grafico de Boxplot com todos os valores validos da coluna remuneracao de todas as cidades
df_empregos['remuneracao'].plot.box(grid = True)

Analise explícita

In [ ]:
# VISUALIZAR OS VALORES DE SALARIOS MIN , MED E MAX DE CADA CIDADE SEPARADAMENTE

# criação de lista vazia para receber os dicionarios de cada cidade
salarios_min_max = []
# obtendo lista com o nome das cidades
l_cidade = df_empregos['cidade'].drop_duplicates()
l_cidade = l_cidade.values
# colocando os valores de cada cidade na lista salarios_min_max
for i3 in range(len(l_cidade)):
  dici_temp = {'cidade': l_cidade[i3],'min':df_empregos[df_empregos['cidade'] == l_cidade[i3]].loc[:,'remuneracao'].min(),'media':df_empregos[df_empregos['cidade'] == l_cidade[i3]].loc[:,'remuneracao'].mean(),'max':df_empregos[df_empregos['cidade'] == l_cidade[i3]].loc[:,'remuneracao'].max()}
  salarios_min_max.append(dici_temp)
# criando dataframe
df_salarios_min_max = pd.DataFrame(salarios_min_max)
df_salarios_min_max

In [ ]:
# GRAFICO COM O VALOR TOTAL DA COLUNA REMUNERAÇÃO PARA CADA CIDADE
ax = df_soma_salario.plot.bar( x='cidade', y='valor', rot=50)

In [ ]:
# EXIBINDO AS QUANTIDADES DE VAGAS POR COMPANHIAS PARA A CIDADE SELECIONADA

# Escolha um numero corespondente a cidade e atribua esse valor a variavel selecao
# (0 -> natal-rn) (1 -> joao-pessoa-pb) (2 -> fortaleza-ce) (3 -> recife-pe) (4 -> salvador-ba)
# (5 -> sao-luis-ma) (6 -> teresina-pi) (7 -> aracaju-se) (8 -> maceio-al)
selecao=0

cidade_selecionada = ['natal-rn','joao-pessoa-pb','fortaleza-ce','recife-pe','salvador-ba','sao-luis-ma','teresina-pi','aracaju-se','maceio-al',]
lista_temporaria = []
# criando outro dataframe sem as vagas que não tem informações sobre a companhia
comp_vagas = df_empregos[df_empregos['cidade'] == cidade_selecionada[selecao]]
comp_vagas = comp_vagas[comp_vagas['companhia'] != '(Confidencial)']
comp_vagas = comp_vagas[comp_vagas['companhia'] != 'Confidencial']

comp_vagas = comp_vagas['companhia']
comp_vagas = comp_vagas.value_counts()
for i4 in range(len(comp_vagas - 1)):
  dici_comp_vagas = {'companhia':comp_vagas.index[i4],'numero_de_vagas':comp_vagas[i4] }
  lista_temporaria.append(dici_comp_vagas)
df_comp_vagas = pd.DataFrame(lista_temporaria)
df_comp_vagas

In [ ]:
# VISUALIZAR VAGAS DE EMPREGO PARA A CIDADE SELECIONADA

# Escolha um numero corespondente a cidade e atribua esse valor a variavel selecao
# (0 -> natal-rn) (1 -> joao-pessoa-pb) (2 -> fortaleza-ce) (3 -> recife-pe) (4 -> salvador-ba)
# (5 -> sao-luis-ma) (6 -> teresina-pi) (7 -> aracaju-se) (8 -> maceio-al)
selecao=0

cidade_selecionada = ['natal-rn','joao-pessoa-pb','fortaleza-ce','recife-pe','salvador-ba','sao-luis-ma','teresina-pi','aracaju-se','maceio-al',]
vagas_cidade = df_empregos[df_empregos['cidade'] == cidade_selecionada[selecao]]
vagas_cidade

In [ ]:
# EXIBIR GRAFICO COM A POSIÇÃO DAS VAGAS DA CIDADE SELECIONADA DE ACORDO COM A FAIXA SALARIAL

# o eixo x corresponde ao id da vaga

# Escolha um numero corespondente a cidade e atribua esse valor a variavel selecao
# (0 -> natal-rn) (1 -> joao-pessoa-pb) (2 -> fortaleza-ce) (3 -> recife-pe) (4 -> salvador-ba)
# (5 -> sao-luis-ma) (6 -> teresina-pi) (7 -> aracaju-se) (8 -> maceio-al)
selecao=0

cidade_selecionada = ['natal-rn','joao-pessoa-pb','fortaleza-ce','recife-pe','salvador-ba','sao-luis-ma','teresina-pi','aracaju-se','maceio-al',]
vagas_cidade = df_empregos[df_empregos['cidade'] == cidade_selecionada[selecao]]
vagas_cidade = vagas_cidade['faixa_salarial']
sns.relplot(data=vagas_cidade)

# Nova seção